In [1]:
import pyomo.environ as pyo
import json
from datetime import timedelta
import polars as pl
from polars import selectors as cs
from polars  import col as c
import os 
import numpy as np
import math
import tqdm
from datetime import timedelta, datetime, timezone
# from optimization_model.optimizaztion_pipeline import first_stage_pipeline
from typing_extensions import Optional
from data_display.input_data_plots import plot_basin_height_volume_table

from data_display.baseline_plots import *
from utility.pyomo_preprocessing import extract_optimization_results, linear_interpolation_for_bound, arange_float, linear_interpolation_using_cols 
from data_federation.input_model import SmallflexInputSchema
from utility.pyomo_preprocessing import *
from config import settings
from utility.general_function import pl_to_dict, pl_to_dict_with_tuple, build_non_existing_dirs, generate_log
from pyomo_models.input_data_preprocessing import (
    generate_baseline_index, generate_clean_timeseries, generate_water_flow_factor, generate_basin_volume_table,
    clean_hydro_power_performance_table, generate_hydro_power_state, split_timestamps_per_sim, generate_second_stage_state
)
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.graph_objs as go
from plotly.graph_objects import Figure

from plotly.subplots import make_subplots

import networkx as nx
from pyomo_models.baseline.baseline_input import BaseLineInput
from pyomo_models.baseline.first_stage.first_stage_pipeline import BaselineFirstStage
from pyomo_models.baseline.second_stage.second_stage_pipeline import BaselineSecondStage
COLORS = px.colors.qualitative.Plotly

log = generate_log(name="test")

In [2]:
os.chdir(os.getcwd().replace("/src", ""))
os.environ['GRB_LICENSE_FILE'] = "/home/ltomasini/gruobi_license/gurobi.lic"


In [3]:
output_file_names: dict[str, str] = json.load(open(settings.OUTPUT_FILE_NAMES))

sim_setting: dict = {"quantile": 0.15, "buffer": 0.3, "powered_volume_enabled": True, "with_penalty": True}
baseline_input = BaseLineInput(
    input_schema_file_name=output_file_names["duckdb_input"],
    real_timestep=timedelta(hours=1),
    year=2023,
    hydro_power_mask = c("name").is_in(["Aegina hydro"]),
    volume_factor=1e-6,
)
baseline_first_stage = BaselineFirstStage(
    baseline_input, timestep=timedelta(days=1), turbine_factor=0.7)
baseline_first_stage.solve_model() 

sim_setting = {"quantile": 0, "buffer": 0.2, "powered_volume_enabled": True, "with_penalty": True}
baseline_second_stage = BaselineSecondStage(
    input_instance=baseline_input, 
    first_stage=baseline_first_stage, 
    timestep=timedelta(days=4), 
    log_solver_info=False,
    time_limit=20,
    **sim_setting
    )
baseline_second_stage.solve_model()


Read and validate tables from small_flex_input_data.db file: 100%|████████████████████████████████████████████████████| 12/12 [00:01<00:00,  9.78it/s]
Solving second stage optimization problem:  36%|███▌      | 33/92 [00:56<01:40,  1.70s/it]

{'B': 0, 'start_basin_volume': 2.037239534957608, 'H': 0, 'remaining_volume': 0.0004748724386257411, 'powered_volume': 0.23633856000000275, 'volume_buffer': 0.19353599999999999, 'volume_max': 18.243199999999998, 'volume_min': 0.056996, 'pos_boundary': 1.9944369749576052, 'neg_boundary': 1.6068901025189797}


In [10]:
baseline_second_stage.data

{'H': {None: [0]},
 'B': {None: [0, 1]},
 'buffer': {None: 0.2},
 'water_factor': {(0, 0): -1, (1, 0): 1},
 'big_m': {None: 1000000.0},
 'min_alpha': {0: 3.126419476944253},
 'max_alpha': {0: 4.900863053910109},
 'volume_factor': {None: 1e-06},
 'spilled_factor': {0: 100.0},
 'T': {None: [1,
   2,
   3,
   4,
   5,
   6,
   7,
   8,
   9,
   10,
   11,
   12,
   13,
   14,
   15,
   16,
   17,
   18,
   19,
   20,
   21,
   22,
   23,
   24,
   25,
   26,
   27,
   28,
   29,
   30,
   31,
   32,
   33,
   34,
   35,
   36,
   37,
   38,
   39,
   40,
   41,
   42,
   43,
   44,
   45,
   46,
   47,
   48,
   49,
   50,
   51,
   52,
   53,
   54,
   55,
   56,
   57,
   58,
   59,
   60,
   61,
   62,
   63,
   64,
   65,
   66,
   67,
   68,
   69,
   70,
   71,
   72,
   73,
   74,
   75,
   76,
   77,
   78,
   79,
   80,
   81,
   82,
   83,
   84,
   85,
   86,
   87,
   88,
   89,
   90,
   91,
   92,
   93,
   94,
   95,
   96]},
 'S_B': {0: [0, 1], 1: [2]},
 'S_H': {0: [0, 1]}

In [9]:
baseline_second_stage.infeasible_constraints

name,index,value,bound
str,list[i32],null,i64
"""basin_volume_evolution""","[1, 0]",null,4
"""basin_volume_evolution""","[1, 1]",null,4
"""basin_volume_evolution""","[2, 0]",null,4
"""basin_volume_evolution""","[2, 1]",null,4
"""basin_volume_evolution""","[3, 0]",null,4
…,…,…,…
"""power_constraint""","[92, 0]",null,4
"""power_constraint""","[93, 0]",null,4
"""power_constraint""","[94, 0]",null,4


In [4]:
baseline_second_stage.powered_volume_enabled = False
baseline_second_stage.solve_one_instance(sim_nb=27)

In [5]:
2.351653 * 3600*120 *1e-6 

1.015914096

In [6]:
2.760282537736097 - 0.421811712 - 0.193536

2.1449348257360974

In [7]:
2.760282537736097 - baseline_second_stage.result_flow.filter(c("sim_nb") == 27)["flow"].sum() * 3600*1e-6

1.9532515881533397

In [8]:
self = baseline_second_stage
print(self.volume_buffer.to_pandas().to_string())

    sim_nb  H  volume_buffer
0        0  0       0.048384
1        1  0       0.193536
2        2  0       0.193536
3        3  0       0.193536
4        4  0       0.193536
5        5  0       0.193536
6        6  0       0.193536
7        7  0       0.193536
8        8  0       0.193536
9        9  0       0.193536
10      10  0       0.193536
11      11  0       0.193536
12      12  0       0.193536
13      13  0       0.193536
14      14  0       0.193536
15      15  0       0.193536
16      16  0       0.193536
17      17  0       0.193536
18      18  0       0.193536
19      19  0       0.193536
20      20  0       0.193536
21      21  0       0.193536
22      22  0       0.193536
23      23  0       0.193536
24      24  0       0.193536
25      25  0       0.193536
26      26  0       0.193536
27      27  0       0.193536
28      28  0       0.193536
29      29  0       0.193536
30      30  0       0.193536
31      31  0       0.193536
32      32  0       0.193536
33      33  0 